In [1]:
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

In [2]:
nfl = pd.read_csv("nfl.csv")
nfl.head()

,third_per,third_per_allowed,TOP,date,first_downs,first_downs_allowed,ha,margin,opp,pass_yards,...,rush_yards,rush_yards_allowed,sacked,sacks,takeaways,team,total_points,total_yards,total_yards_allowed,turnovers
0,0.307692,0.285714,28.350000,9/10/2009,18,19,away,-3,PIT,234,...,86,36,1,4,3,TEN,3,320,357,2
1,0.285714,0.307692,36.183333,9/10/2009,19,18,home,3,TEN,321,...,36,86,4,1,2,PIT,3,357,320,3
2,0.363636,0.400000,29.116667,9/13/2009,16,19,away,-12,ATL,163,...,96,68,4,2,0,MIA,26,259,281,4
3,0.400000,0.363636,30.883333,9/13/2009,19,16,home,12,MIA,213,...,68,96,2,4,4,ATL,26,281,259,0
4,0.250000,0.333333,26.550000,9/13/2009,10,16,away,5,CIN,227,...,75,86,3,3,2,DEN,19,302,307,0


In [3]:
nfl.columns

Index(['third_per', 'third_per_allowed', 'TOP', 'date', 'first_downs',
       'first_downs_allowed', 'ha', 'margin', 'opp', 'pass_yards',
       'pass_yards_allowed', 'penalty_yards', 'plays', 'points',
       'points_allowed', 'result', 'rush_yards', 'rush_yards_allowed',
       'sacked', 'sacks', 'takeaways', 'team', 'total_points', 'total_yards',
       'total_yards_allowed', 'turnovers'],
      dtype='object')

In [4]:
teams = nfl['team'].unique().tolist()
teams.remove("ARI")
teams  

['TEN',
 'PIT',
 'MIA',
 'ATL',
 'DEN',
 'CIN',
 'MIN',
 'CLE',
 'JAX',
 'IND',
 'DET',
 'NOS',
 'DAL',
 'TBB',
 'PHI',
 'CAR',
 'KCC',
 'BAL',
 'NYJ',
 'HOU',
 'WAS',
 'NYG',
 'SFO',
 'LAR',
 'SEA',
 'CHI',
 'GBP',
 'BUF',
 'NEP',
 'LAC',
 'OAK']

In [5]:
margin_model = smf.ols(formula='margin ~ third_per + third_per_allowed + TOP + first_downs + first_downs_allowed + ha + opp + pass_yards + pass_yards_allowed + penalty_yards + plays + rush_yards + rush_yards_allowed + sacked + sacks + takeaways + team + total_yards + total_yards_allowed + turnovers', data=nfl)
margin_res = margin_model.fit()
print(margin_res.summary())

                            OLS Regression Results                            
Dep. Variable:                 margin   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     237.4
Date:                Thu, 28 Jun 2018   Prob (F-statistic):               0.00
Time:                        20:03:55   Log-Likelihood:                -16029.
No. Observations:                4806   AIC:                         3.222e+04
Df Residuals:                    4727   BIC:                         3.273e+04
Df Model:                          78                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              29.4673    

In [6]:
print(margin_res.params)

Intercept              29.467291
ha[T.home]              1.623563
opp[T.ATL]             -1.027883
opp[T.BAL]             -0.374861
opp[T.BUF]              0.590217
opp[T.CAR]              1.189104
opp[T.CHI]             -0.145734
opp[T.CIN]             -0.762804
opp[T.CLE]              1.394555
opp[T.DAL]             -1.369693
opp[T.DEN]              0.508341
opp[T.DET]              0.039180
opp[T.GBP]             -1.718302
opp[T.HOU]              2.008900
opp[T.IND]             -0.014422
opp[T.JAX]              1.963263
opp[T.KCC]             -0.245553
opp[T.LAC]             -1.291462
opp[T.LAR]              1.455036
opp[T.MIA]             -0.428902
opp[T.MIN]              0.111915
opp[T.NEP]             -3.394347
opp[T.NOS]             -1.199937
opp[T.NYG]              1.183660
opp[T.NYJ]              1.041092
opp[T.OAK]              1.948242
opp[T.PHI]              0.488784
opp[T.PIT]             -1.807886
opp[T.SEA]             -1.133226
opp[T.SFO]              0.070862
          

In [7]:
ari = 0
opp_ari = 0
for tm in teams:
    r_team = "team[T." + tm + "]"
    o_team = "opp[T." + tm + "]"
    team_score = margin_res.params[r_team]
    opp_score = margin_res.params[o_team]
    ari = ari + team_score
    opp_ari = opp_ari + opp_score   
margin_ari_score = -1*ari/len(teams)
margin_ari_opp = -1*opp_ari/len(teams)
print (margin_ari_score, margin_ari_opp)

0.258019840445 -0.104523161855


In [8]:
nfl['pred_margin'] = ""

In [9]:
margins = []
for key, row in nfl.iterrows():
    if row.team == "ARI":
        team_coeff = margin_ari_score
    else:
        res_team = "team[T." + row.team + "]"
        team_coeff = margin_res.params[res_team]
    if row.opp == "ARI":
        opp_coeff = margin_ari_opp
    else:
        res_opp = "team[T." + row.opp + "]"
        opp_coeff = margin_res.params[res_opp]
#     print(team_coeff, opp_coeff)
    if row.ha == "away":
        ha_coeff = margin_res.params["ha[T.home]"]*-1
    else:
        ha_coeff = margin_res.params["ha[T.home]"]
#     print(ha_coeff)
    margin_predict = margin_res.params.Intercept + margin_res.params.third_per*row['third_per'] + margin_res.params.third_per_allowed*row['third_per_allowed'] + margin_res.params.TOP*row['TOP'] + margin_res.params.first_downs*row['first_downs'] + margin_res.params.first_downs_allowed*row['first_downs_allowed'] + margin_res.params.pass_yards*row['pass_yards'] + margin_res.params.pass_yards_allowed*row['pass_yards_allowed'] + margin_res.params.penalty_yards*row['penalty_yards'] + margin_res.params.plays*row['plays'] + margin_res.params.rush_yards*row['rush_yards'] + margin_res.params.rush_yards_allowed*row['rush_yards_allowed'] + margin_res.params.sacked*row['sacked'] + margin_res.params.sacks*row['sacks'] + margin_res.params.takeaways*row['takeaways'] + margin_res.params.total_yards*row['total_yards'] + margin_res.params.total_yards_allowed*row['total_yards_allowed'] + margin_res.params.turnovers*row['turnovers'] + ha_coeff + team_coeff + opp_coeff
    print(key, margin_predict)
    margins.append(margin_predict)

0 5.5697573639
1 -8.76042019429
2 -15.8784805612
3 19.0918968684
4 8.21400463062
5 -0.423524847794
6 13.7370638525
7 -15.1372310632
8 -4.2342219322
9 -1.52838501451
10 -26.9910976321
11 28.3800062209
12 -3.60106593772
13 1.63341365001
14 26.5854034705
15 -33.8937193608
16 -15.9600759662
17 19.3035578363
18 14.7523223788
19 -21.5701739009
20 -5.32726294911
21 5.96549047142
22 -2.10782139254
23 -0.673931967219
24 -15.9306612187
25 13.7334055469
26 -11.4847243764
27 11.1810036249
28 -6.5739748355
29 14.5636280899
30 -6.41773757626
31 0.691194512465
32 -1.71124651739
33 -3.5020198923
34 6.65195445074
35 -6.53613061339
36 1.91604562033
37 -0.91659036233
38 -5.62165526663
39 -3.62396886638
40 -9.14556095682
41 1.14476539323
42 0.898085567603
43 1.99272717833
44 13.2324238116
45 -11.6884085269
46 -7.68994671077
47 -1.10047126862
48 7.84819359967
49 -8.58047056135
50 -11.3232034772
51 10.7068024294
52 -19.9127799486
53 8.9259955291
54 -5.08083392625
55 4.06231965224
56 3.1174193587
57 -0.61072

719 25.1650405378
720 3.76693674998
721 2.94508320568
722 -0.362513665378
723 0.593609906
724 6.51941149496
725 -8.07387982989
726 12.3438686334
727 -10.1037392474
728 1.76953605925
729 -0.426227401402
730 -3.75238939889
731 -7.81214867631
732 -16.1508214297
733 9.77458116832
734 30.9128870781
735 -32.4317974922
736 4.00078963149
737 0.0824339030906
738 -5.57383866047
739 8.92482156928
740 7.55130188638
741 -7.21138641049
742 -21.1144750094
743 11.5161077429
744 -19.3389852815
745 2.78033165108
746 10.8950781928
747 -13.265927514
748 -2.74566436806
749 -0.466736667177
750 -0.745553517277
751 -5.00140764597
752 -15.9084971207
753 4.46428783052
754 -9.67377025032
755 13.2758765011
756 -12.4927822997
757 2.58655249783
758 2.87645237819
759 -1.60961135646
760 -34.9705999354
761 28.7381879313
762 -3.38770642687
763 12.5856016916
764 -0.887054449229
765 5.41538984911
766 -16.752363604
767 10.6397255825
768 -5.2748542163
769 6.53131151348
770 -10.3391926094
771 17.20278503
772 0.865423580902


1218 7.30554425507
1219 -4.00830989735
1220 -6.6029268779
1221 5.84176557851
1222 -6.10103507367
1223 5.70460741703
1224 -14.1150256935
1225 11.7952015305
1226 -11.3159869719
1227 11.5276099644
1228 -15.2159250354
1229 11.1045967348
1230 -10.3370684291
1231 1.23397506905
1232 15.4439627439
1233 -15.3346708371
1234 -14.7294240353
1235 10.6172035925
1236 -12.2388733676
1237 0.540116765195
1238 -0.419077511382
1239 3.22685622963
1240 -16.2971488713
1241 12.1093732405
1242 1.61384701162
1243 2.03569149452
1244 -11.1238659507
1245 9.86694528274
1246 -11.277905583
1247 14.5536694833
1248 29.8366379799
1249 -35.0448349749
1250 8.38129597352
1251 -18.7742082512
1252 -18.8931928782
1253 9.19021770681
1254 -12.0347529316
1255 5.56437585705
1256 -17.4378887333
1257 8.04913790008
1258 1.87391753479
1259 -4.5086627255
1260 -2.34104983852
1261 -2.86931117578
1262 7.81155594086
1263 -4.30297810808
1264 10.5232188555
1265 -21.9296139675
1266 -0.510023453059
1267 4.54080980517
1268 -18.2498747206
1269 

1716 2.35451289186
1717 -6.35850770785
1718 11.7576180977
1719 -7.82035559036
1720 5.84336337117
1721 -13.9706487003
1722 -0.646219365213
1723 0.162986600992
1724 -9.52213578654
1725 9.82197737638
1726 17.2113663771
1727 -12.1448155638
1728 -11.8028841613
1729 4.42176891815
1730 6.22741416141
1731 -2.43172532343
1732 -21.595779406
1733 22.0669533699
1734 -7.41932300347
1735 -3.06582387294
1736 -12.4472818536
1737 12.858046433
1738 7.12272823763
1739 -8.5199221511
1740 0.974459373284
1741 2.7265545896
1742 21.4256198365
1743 -31.64730964
1744 5.03026479452
1745 -3.28244981467
1746 -17.3048799087
1747 7.80428662885
1748 -32.2954293036
1749 41.0760715064
1750 -13.7554820589
1751 13.1819148694
1752 -3.80851241244
1753 2.23123937872
1754 9.61968443264
1755 -12.7870877172
1756 -3.44964566597
1757 -4.2379483357
1758 -10.3394946287
1759 8.57887361486
1760 -28.6032312926
1761 26.2999929691
1762 -19.9552388933
1763 17.2089303772
1764 5.81616652057
1765 -4.30161538668
1766 -1.42861854544
1767 0.1

2215 16.0517898373
2216 -26.5767352222
2217 22.1779770919
2218 -12.6444564621
2219 8.19657348585
2220 5.12534028403
2221 -3.18300366423
2222 -23.1433155367
2223 24.2349445371
2224 17.4678488969
2225 -14.0684945238
2226 -20.3680092239
2227 2.46392173956
2228 11.4098044361
2229 -10.9114907903
2230 -10.1189522618
2231 11.4081689292
2232 12.2122310337
2233 -22.4023352273
2234 -6.24553259529
2235 -4.48666702484
2236 -14.9990551729
2237 9.374804975
2238 -16.0144341593
2239 8.57094749076
2240 -10.2145844697
2241 0.553372326805
2242 -9.61517729139
2243 12.3327092226
2244 23.9542597362
2245 -24.3588372515
2246 -20.5524396342
2247 13.1863967575
2248 -9.81447229953
2249 9.25849880248
2250 -26.0071713381
2251 19.0550827695
2252 -12.5182978526
2253 13.3761161539
2254 8.30028004147
2255 -18.1342717012
2256 -9.02461678688
2257 9.94415263975
2258 7.57821330346
2259 -4.08390563679
2260 -15.4605858704
2261 24.4155541999
2262 -10.7421970192
2263 -0.0262388200168
2264 -19.4682913472
2265 10.5558018861
226

2715 -0.33761129313
2716 -25.7576037245
2717 23.1594462999
2718 -13.7297938404
2719 9.45062675855
2720 -6.66935592368
2721 4.77998804415
2722 -6.45322442312
2723 10.5705007727
2724 12.6828997197
2725 -18.4298229694
2726 -10.8836329558
2727 3.3273716546
2728 2.78783608801
2729 -0.159055369848
2730 9.76335006687
2731 -11.790055051
2732 6.80102229508
2733 -9.2252336458
2734 -35.4806396522
2735 30.9759204229
2736 -0.823641603572
2737 -7.14214962419
2738 -1.01006861815
2739 -3.50491870532
2740 -1.6884194437
2741 -11.1448677073
2742 -15.8898971473
2743 13.5954252407
2744 -16.4279404201
2745 16.8322258775
2746 -10.5796366229
2747 14.6127296804
2748 -2.01957440183
2749 7.8269382893
2750 -0.342427793242
2751 -1.02263801996
2752 19.7744177735
2753 -19.556267336
2754 -7.27793830106
2755 5.87138754337
2756 -5.92398670017
2757 3.98965588107
2758 -10.7619521358
2759 8.62115818066
2760 -5.09354775282
2761 -1.46946841216
2762 13.2771816279
2763 -14.3984836939
2764 -6.56289772601
2765 -5.49722989263
27

3214 -3.27632884394
3215 -2.76123105233
3216 4.98806616661
3217 -16.6127454482
3218 -7.07062420711
3219 3.1544562579
3220 -14.3516276284
3221 16.4645311691
3222 -2.86733999011
3223 9.23026663943
3224 15.2627423673
3225 -17.7774693437
3226 16.5721785869
3227 -16.5312617085
3228 -7.45825146898
3229 9.33807616632
3230 -3.09790319029
3231 -0.603055728797
3232 -2.70197461763
3233 0.193590473478
3234 -14.8077932574
3235 20.6154974102
3236 16.7330001861
3237 -12.4978012091
3238 -22.4877289545
3239 -1.99633112885
3240 -19.7748337767
3241 17.7214624478
3242 -0.393382580959
3243 -7.96594681714
3244 -15.1112125477
3245 10.3250496037
3246 11.4558070163
3247 -12.7487697899
3248 10.3158005766
3249 -8.83445378617
3250 -7.29901307366
3251 11.8088258103
3252 -17.6808445242
3253 13.3002247297
3254 -0.740424056978
3255 -7.95784802786
3256 -12.7168598012
3257 5.90847758031
3258 -11.168704753
3259 0.26732599227
3260 -8.43335421242
3261 5.99328271662
3262 5.07338644942
3263 -6.56279880228
3264 -3.4411890327

3714 5.46766371636
3715 -1.94390424916
3716 15.274578888
3717 -20.2249634533
3718 3.52354797695
3719 -9.17608223621
3720 3.67072925563
3721 -1.03453007001
3722 1.89773320115
3723 -8.5175060349
3724 -5.89719117632
3725 7.12802231068
3726 3.99749808246
3727 -4.60371543593
3728 -10.2767538281
3729 8.0371776326
3730 -2.58903213294
3731 3.17195255273
3732 -7.50682078805
3733 6.16851343065
3734 -36.9996646365
3735 36.7278429844
3736 -8.10379213805
3737 2.79886914842
3738 1.024348639
3739 -4.92863027835
3740 -0.137100825875
3741 -2.95026008383
3742 3.62068330702
3743 -10.1184480049
3744 -7.58219816163
3745 12.272917603
3746 -3.27377711516
3747 7.92056131315
3748 0.564487705757
3749 -1.25507035043
3750 4.29030681171
3751 -7.82674363441
3752 -1.94188447395
3753 6.99618535745
3754 -13.8398821375
3755 2.3976840433
3756 6.3808059184
3757 -11.2158304532
3758 -2.37057627746
3759 -1.31489149098
3760 -9.32319712636
3761 1.02489529301
3762 1.65331068467
3763 -2.02966613077
3764 -6.41735725157
3765 11.1

4215 22.3443506014
4216 -21.3061218233
4217 22.7327167472
4218 -7.80563256119
4219 13.6645096712
4220 -15.4865447209
4221 2.91038089291
4222 -10.8508453114
4223 1.56784007021
4224 3.097111585
4225 -4.75063566357
4226 14.7368192495
4227 -10.1316270228
4228 -22.5915139494
4229 25.2631825927
4230 -19.6981080416
4231 20.6200865156
4232 -15.3448445659
4233 12.5640714799
4234 -1.66512074701
4235 -3.15089135261
4236 -12.7165646668
4237 11.7632995123
4238 9.82059925644
4239 -15.8573299502
4240 19.5492193563
4241 -22.5012525063
4242 -21.0700774058
4243 18.5991946254
4244 9.73051643576
4245 -4.68390696127
4246 16.2163693594
4247 -11.479925675
4248 6.64338594479
4249 -6.88708438673
4250 -31.2133080017
4251 15.4546772591
4252 -15.053232282
4253 16.5787305744
4254 -13.8977710649
4255 18.9016557903
4256 -12.3977262982
4257 10.8954572488
4258 -11.9074601828
4259 19.5190862788
4260 -11.6179927935
4261 11.245631589
4262 0.307805155196
4263 4.35626196322
4264 12.9488084914
4265 -9.31249520011
4266 -22.6

In [10]:
print (len(margins))

4806


In [11]:
avg_margins = []
count = 0
for x in np.arange(len(margins)/2):
    away_marg = float(margins[int(x*2)]) + float(margins[int((x*2)+1)])
    home_marg = -1*away_marg
    avg_margins.append(away_marg)
    avg_margins.append(home_marg)
nfl['pred_margin'] = avg_margins
    

In [12]:
nfl[['margin', "pred_margin"]]

,margin,pred_margin
0,-3,-3.190663
1,3,3.190663
2,-12,3.213416
3,12,-3.213416
4,5,7.790480
5,-5,-7.790480
6,14,-1.400167
7,-14,1.400167
8,-2,-5.762607
9,2,5.762607


In [13]:
count = 0
for g in np.arange(len(avg_margins)/2):
    m = avg_margins[int(g*2)]
#     print(m)
    if m < 0:
        count = count + 1
# print(count)
print(f"Percent of Home Favorites: {count/(len(avg_margins)/2)}")


Percent of Home Favorites: 0.640033291718685


In [14]:
residuals = []
for key, row in nfl.iterrows():
    residual = row['pred_margin'] - row['margin']
    residuals.append(residual)
diffs = []
for r in np.arange(len(residuals)/2):
    diff = residuals[int(r*2)]
    diffs.append(diff)
# print(diffs)
print(np.average(diffs))
    
    

0.325725460269


In [15]:
total_model = smf.ols(formula='total_points ~ third_per + third_per_allowed + TOP + first_downs + first_downs_allowed + ha + opp + pass_yards + pass_yards_allowed + penalty_yards + plays + rush_yards + rush_yards_allowed + sacked + sacks + takeaways + team + total_yards + total_yards_allowed + turnovers', data=nfl)
total_res = total_model.fit()
print(total_res.summary())

                            OLS Regression Results                            
Dep. Variable:           total_points   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     23.99
Date:                Thu, 28 Jun 2018   Prob (F-statistic):          3.16e-280
Time:                        20:04:07   Log-Likelihood:                -19533.
No. Observations:                4806   AIC:                         3.922e+04
Df Residuals:                    4727   BIC:                         3.974e+04
Df Model:                          78                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              39.3515    

In [16]:
print(total_res.params)

Intercept              39.351451
ha[T.home]             -0.735652
opp[T.ATL]             -2.086297
opp[T.BAL]             -0.233090
opp[T.BUF]              0.365906
opp[T.CAR]              1.568192
opp[T.CHI]             -0.570478
opp[T.CIN]             -0.862933
opp[T.CLE]             -2.889062
opp[T.DAL]             -2.861613
opp[T.DEN]              2.653180
opp[T.DET]             -0.621008
opp[T.GBP]              2.306390
opp[T.HOU]             -0.358277
opp[T.IND]              0.897084
opp[T.JAX]              0.551890
opp[T.KCC]             -2.059157
opp[T.LAC]             -1.725887
opp[T.LAR]             -0.609776
opp[T.MIA]             -0.902181
opp[T.MIN]              0.105825
opp[T.NEP]              2.387007
opp[T.NOS]              0.089922
opp[T.NYG]              1.002689
opp[T.NYJ]             -0.409488
opp[T.OAK]              1.422782
opp[T.PHI]              4.434107
opp[T.PIT]             -1.299627
opp[T.SEA]              1.037289
opp[T.SFO]             -0.015943
          

In [17]:
ari = 0
opp_ari = 0
for tm in teams:
    r_team = "team[T." + tm + "]"
    o_team = "opp[T." + tm + "]"
    team_score = total_res.params[r_team]
    opp_score = total_res.params[o_team]
    ari = ari + team_score
    opp_ari = opp_ari + opp_score   
total_ari_score = -1*ari/len(teams)
total_ari_opp = -1*opp_ari/len(teams)
print (total_ari_score, total_ari_opp)

-1.25125413092 0.0861553904417


In [18]:
nfl['pred_total'] = ""

In [19]:
totals = []
for key, row in nfl.iterrows():
    if row.team == "ARI":
        team_coeff = total_ari_score
    else:
        res_team = "team[T." + row.team + "]"
        team_coeff = total_res.params[res_team]
    if row.opp == "ARI":
        opp_coeff = total_ari_opp
    else:
        res_opp = "team[T." + row.opp + "]"
        opp_coeff = total_res.params[res_opp]
#     print(team_coeff, opp_coeff)
    if row.ha == "away":
        ha_coeff = total_res.params["ha[T.home]"]*-1
    else:
        ha_coeff = total_res.params["ha[T.home]"]
#     print(ha_coeff)
    total_predict = total_res.params.Intercept + total_res.params.third_per*row['third_per'] + total_res.params.third_per_allowed*row['third_per_allowed'] + total_res.params.TOP*row['TOP'] + total_res.params.first_downs*row['first_downs'] + total_res.params.first_downs_allowed*row['first_downs_allowed'] + total_res.params.pass_yards*row['pass_yards'] + total_res.params.pass_yards_allowed*row['pass_yards_allowed'] + total_res.params.penalty_yards*row['penalty_yards'] + total_res.params.plays*row['plays'] + total_res.params.rush_yards*row['rush_yards'] + total_res.params.rush_yards_allowed*row['rush_yards_allowed'] + total_res.params.sacked*row['sacked'] + total_res.params.sacks*row['sacks'] + total_res.params.takeaways*row['takeaways'] + total_res.params.total_yards*row['total_yards'] + total_res.params.total_yards_allowed*row['total_yards_allowed'] + total_res.params.turnovers*row['turnovers'] + ha_coeff + team_coeff + opp_coeff
    print(key, total_predict)
    totals.append(total_predict)

0 45.979484181
1 35.4455407431
2 34.879195
3 35.2186608243
4 38.5311930356
5 32.3820187022
6 33.9151085308
7 32.9945754788
8 41.9879196148
9 38.3609770765
10 50.8026526928
11 56.5482721547
12 51.2235511059
13 46.2528619523
14 43.3565274428
15 31.5042915049
16 41.4123392524
17 39.2744799909
18 41.0350380515
19 45.6553552721
20 51.0088071258
21 36.2503288647
22 28.862320821
23 31.966108686
24 40.7690935668
25 46.0374986998
26 32.9193532619
27 40.8358682542
28 55.9264417152
29 44.2371746244
30 52.9141126031
31 46.3824612518
32 58.8276976462
33 52.0772136646
34 41.0400818569
35 33.1340966774
36 43.320048064
37 52.598080175
38 43.8046871897
39 58.1027510645
40 41.362262526
41 32.2403471733
42 34.4684054381
43 33.9289038654
44 61.9748666681
45 54.1582868621
46 44.6393263108
47 34.811516813
48 47.9705745339
49 46.5210935892
50 37.6893861659
51 31.5263106789
52 43.3069001176
53 53.8484059121
54 43.5483720443
55 40.1655147114
56 52.0427884228
57 43.6859060589
58 35.1278826847
59 45.6627790293
6

622 40.9354726676
623 44.8505128201
624 53.0256488718
625 53.4115973457
626 57.1789789978
627 48.5599314123
628 43.3441923537
629 54.1015616327
630 36.9385841711
631 42.1344942648
632 36.0442111637
633 34.5599497677
634 39.7171407869
635 42.7651724977
636 50.3432359814
637 61.0828140224
638 49.3389748753
639 41.8109620113
640 36.8274733996
641 42.0338405007
642 47.5244610585
643 40.3743794399
644 38.0792111765
645 34.8676900419
646 59.7343237025
647 54.5229208614
648 55.4612037763
649 52.7828662815
650 46.7726099565
651 39.3059680202
652 34.4747803893
653 38.6376405571
654 28.1734744928
655 30.8531837832
656 49.3605269533
657 43.5523720596
658 28.9398931273
659 22.3445044025
660 37.7604149227
661 34.3037779694
662 38.8938463129
663 36.11026415
664 49.4598773129
665 48.5520015797
666 41.1887046921
667 44.2861949259
668 41.523732686
669 51.0276951305
670 39.458964987
671 39.7092893238
672 54.1288895775
673 47.0574328747
674 41.9956201967
675 48.0138691345
676 40.6709617137
677 31.8032865

1124 68.2176827508
1125 69.5978141244
1126 36.2001839542
1127 40.9565748006
1128 55.2840793636
1129 51.5147175398
1130 41.6785756509
1131 42.9789753497
1132 33.0306227943
1133 44.526282229
1134 54.3265565332
1135 43.9097016284
1136 56.7674909957
1137 62.9409667238
1138 65.4292444531
1139 64.0167837749
1140 32.2855719491
1141 35.3681068859
1142 27.8385873367
1143 30.7744135056
1144 35.942781077
1145 39.3207575641
1146 46.3387710383
1147 43.1717636059
1148 48.5309696729
1149 43.6259866809
1150 50.1544545389
1151 53.774941829
1152 42.6607938351
1153 38.6807080513
1154 49.1536447875
1155 32.6097325621
1156 44.3697203162
1157 47.3604441658
1158 29.2576841817
1159 33.999270729
1160 41.492625969
1161 43.0860601252
1162 43.2809030981
1163 34.1472584291
1164 49.3288369343
1165 45.6788248553
1166 46.9430894498
1167 51.4040523322
1168 52.8043679776
1169 41.0301593636
1170 36.5576506888
1171 39.6177658105
1172 38.5401532234
1173 48.0988451218
1174 46.289852011
1175 36.7705725544
1176 27.2773803327

1620 54.4966929606
1621 50.720799534
1622 45.3792161982
1623 30.6286594546
1624 29.8334454987
1625 36.4082672211
1626 43.279786281
1627 46.4392699709
1628 37.2950044126
1629 51.0246165455
1630 35.962760146
1631 53.3483123449
1632 39.4324313303
1633 41.1388374016
1634 34.8570877253
1635 36.2737977884
1636 52.342851632
1637 43.9362081825
1638 51.0058268858
1639 44.5790086878
1640 53.078616549
1641 51.7511350685
1642 42.1859907277
1643 37.2952831888
1644 33.3902165152
1645 39.2779731048
1646 60.1159594821
1647 60.8538831615
1648 29.4638966522
1649 39.1945094152
1650 54.5568925358
1651 60.7592769662
1652 47.5940354298
1653 50.4741714726
1654 51.9778133243
1655 53.4065688563
1656 41.4117121995
1657 34.1927182913
1658 44.6837794856
1659 36.2355328982
1660 43.3758140571
1661 35.7163031216
1662 36.1358245641
1663 48.0135518242
1664 47.6607244157
1665 40.3177431657
1666 53.7941021435
1667 59.9105018653
1668 34.260596411
1669 43.8709278061
1670 51.4990697263
1671 54.0517105624
1672 58.0997783286

2120 36.517542865
2121 34.9954414041
2122 43.5139768004
2123 45.3148210684
2124 61.8708111846
2125 57.329295648
2126 60.4231550218
2127 53.511759294
2128 47.5211686643
2129 55.7838257016
2130 55.0553690715
2131 53.0427082867
2132 50.0563607084
2133 46.8900432855
2134 45.0928354579
2135 51.9242945931
2136 47.9650094296
2137 62.7316098217
2138 27.5347128676
2139 32.1856035304
2140 34.715488109
2141 33.6008807686
2142 41.9407233904
2143 48.0213862424
2144 52.3653904122
2145 42.8546857981
2146 45.4567486191
2147 44.4736908371
2148 29.8883298297
2149 22.5689879875
2150 35.9576652997
2151 38.4720653995
2152 44.8901305552
2153 53.6065946382
2154 51.8514502109
2155 39.2003202889
2156 55.6294317091
2157 51.5344121732
2158 53.2359295371
2159 47.2332211526
2160 66.1441069815
2161 45.7019301673
2162 65.1190692251
2163 46.3334897185
2164 54.0595135986
2165 52.1006913842
2166 44.7754903904
2167 48.7168517862
2168 29.6822481417
2169 29.6581104764
2170 33.7747314322
2171 38.2599556561
2172 52.16157819

2641 30.5322926158
2642 50.7401634927
2643 36.9619451551
2644 54.3871620792
2645 47.9857863097
2646 54.1112049668
2647 43.5666675254
2648 61.9927854541
2649 73.8351353387
2650 48.6580990522
2651 45.0338687278
2652 44.3188842943
2653 44.2270673178
2654 45.5806742675
2655 35.9678142713
2656 41.3926574438
2657 34.3586115057
2658 47.9407333565
2659 50.4575873846
2660 44.9475008815
2661 46.3277757167
2662 45.9646532752
2663 43.1571587279
2664 56.3121486503
2665 48.5154496975
2666 39.0199127351
2667 34.3831828466
2668 50.1943617096
2669 43.2179578677
2670 47.5188758823
2671 42.9239251695
2672 66.8045479151
2673 64.573438974
2674 44.5399368145
2675 35.2342663324
2676 50.1987950859
2677 44.9050296965
2678 42.5597387523
2679 47.5026985327
2680 39.877373125
2681 39.8146558829
2682 39.8446876434
2683 47.7719219601
2684 47.5574986245
2685 52.0500159475
2686 44.5065964968
2687 43.1101242336
2688 39.4466612174
2689 38.0430593854
2690 39.2453150553
2691 45.982285626
2692 36.9617263958
2693 46.6994060

3120 57.4936042725
3121 50.8411130248
3122 51.3296091115
3123 54.0839697497
3124 58.6523490564
3125 62.6865358475
3126 35.4668700838
3127 26.3138868133
3128 42.6917187775
3129 39.9510207723
3130 40.6448846564
3131 39.8936053402
3132 44.5579357569
3133 48.7528065389
3134 33.0666334828
3135 36.0003873162
3136 40.4618493381
3137 40.725983776
3138 41.2241525576
3139 29.321399152
3140 52.7578288993
3141 62.4570456073
3142 44.0843358761
3143 39.5018780392
3144 45.4847181475
3145 45.8329545108
3146 47.2096512385
3147 36.0419283067
3148 53.7560489335
3149 46.0704606442
3150 32.4957367004
3151 40.3152790762
3152 53.0448180394
3153 49.2765832173
3154 29.9073310141
3155 30.2445355898
3156 42.551859885
3157 30.8326111778
3158 35.4497124031
3159 32.0247737786
3160 45.3235154106
3161 55.3215342905
3162 28.8625785678
3163 36.7850546413
3164 35.3645184027
3165 29.380023058
3166 65.2558971294
3167 47.6690597076
3168 44.6740668108
3169 38.9264345894
3170 41.6954484484
3171 35.0479193955
3172 54.62918989

3619 46.2072334748
3620 52.6328887372
3621 44.8352190064
3622 44.2433313511
3623 40.2951961504
3624 46.7936204611
3625 44.0768418652
3626 39.5754592785
3627 44.5184505378
3628 35.1659365981
3629 37.7977996949
3630 64.2411834501
3631 50.9926783188
3632 37.0752095793
3633 40.9610803674
3634 53.0874381782
3635 54.678749585
3636 37.1967901248
3637 37.6081009442
3638 40.3729557692
3639 45.5269987511
3640 45.8446424621
3641 45.2258366707
3642 52.3802660825
3643 54.3277333045
3644 44.0707296305
3645 39.894044223
3646 34.3160680665
3647 31.9234924532
3648 57.745476668
3649 58.8343048438
3650 58.9313490036
3651 49.045371739
3652 33.827025475
3653 33.3447460162
3654 47.9011807071
3655 51.3366671894
3656 38.6353484543
3657 42.0216416545
3658 39.4977270917
3659 38.9261220407
3660 37.5193683958
3661 34.8331235302
3662 47.0662104894
3663 37.6566437191
3664 46.354913877
3665 38.8182299382
3666 43.7399189052
3667 47.0476113853
3668 38.910136253
3669 48.5475602738
3670 50.6630228688
3671 38.8500029781


4119 57.3161571654
4120 49.0982918295
4121 47.6593252195
4122 38.0958126485
4123 39.5621602494
4124 40.0855252749
4125 53.8487724119
4126 41.9984830636
4127 35.2366394804
4128 54.3610368037
4129 41.5006445296
4130 44.6897338037
4131 38.3412328272
4132 42.0832519191
4133 36.3477168423
4134 34.3615318059
4135 34.4847409015
4136 48.2151581623
4137 43.0734720303
4138 42.1377734129
4139 51.9360427531
4140 43.8330648594
4141 45.2449735664
4142 31.4320470049
4143 42.8698574156
4144 36.2453532466
4145 40.6979545216
4146 45.8345712994
4147 38.146014538
4148 47.8042219171
4149 50.8990588594
4150 30.6400808631
4151 26.9981820439
4152 46.6974258831
4153 51.6959048915
4154 31.6014851991
4155 32.8869801183
4156 48.6141705972
4157 39.8900312316
4158 55.9104741292
4159 62.3989120536
4160 39.869874312
4161 31.4043598222
4162 40.3941082194
4163 44.5101731136
4164 39.397241433
4165 48.3381764265
4166 46.6323796907
4167 38.6991196463
4168 39.7517704762
4169 43.6044436836
4170 47.9794501943
4171 51.6696448

In [20]:
avg_totals = []
count = 0
for x in np.arange(len(totals)/2):
    total = (float(totals[int(x*2)]) + float(totals[int((x*2)+1)]))/2
#     print(total)
    avg_totals.append(total)
    avg_totals.append(total)
nfl['pred_total'] = avg_totals

In [21]:
nfl[['total_points', "pred_total"]]

,total_points,pred_total
0,3,40.712512
1,3,40.712512
2,26,35.048928
3,26,35.048928
4,19,35.456606
5,19,35.456606
6,54,33.454842
7,54,33.454842
8,26,40.174448
9,26,40.174448


In [22]:
residuals = []
for key, row in nfl.iterrows():
    residual = row['pred_total'] - row['total_points']
    residuals.append(residual)
diffs = []
for r in np.arange(len(residuals)/2):
    diff = residuals[int(r*2)]
    diffs.append(diff)
# print(diffs)
print(np.average(diffs))

1.59974568108


In [23]:
nfl.head()

,third_per,third_per_allowed,TOP,date,first_downs,first_downs_allowed,ha,margin,opp,pass_yards,...,sacked,sacks,takeaways,team,total_points,total_yards,total_yards_allowed,turnovers,pred_margin,pred_total
0,0.307692,0.285714,28.350000,9/10/2009,18,19,away,-3,PIT,234,...,1,4,3,TEN,3,320,357,2,-3.190663,40.712512
1,0.285714,0.307692,36.183333,9/10/2009,19,18,home,3,TEN,321,...,4,1,2,PIT,3,357,320,3,3.190663,40.712512
2,0.363636,0.400000,29.116667,9/13/2009,16,19,away,-12,ATL,163,...,4,2,0,MIA,26,259,281,4,3.213416,35.048928
3,0.400000,0.363636,30.883333,9/13/2009,19,16,home,12,MIA,213,...,2,4,4,ATL,26,281,259,0,-3.213416,35.048928
4,0.250000,0.333333,26.550000,9/13/2009,10,16,away,5,CIN,227,...,3,3,2,DEN,19,302,307,0,7.790480,35.456606


In [24]:
nfl['pred_pf'] = ""
nfl['pred_pa'] = ""
pred_pfs = []
pred_pas = []

for key, row in nfl.iterrows():
    pred_margin = row['pred_margin']
    pred_total = row['pred_total']

    a = np.array([[1,1], [1,-1]])
    b = np.array([[pred_total], [pred_margin]])

    points = np.linalg.solve(a,b)
    pf = (points[0][0])
    pa = (points[1][0])
    pred_pfs.append(pf)
    pred_pas.append(pa)
nfl['pred_pf'] = pred_pfs
nfl['pred_pa'] = pred_pas
#     print (f'{pf} - {pa}')

In [25]:
nfl.head()

,third_per,third_per_allowed,TOP,date,first_downs,first_downs_allowed,ha,margin,opp,pass_yards,...,takeaways,team,total_points,total_yards,total_yards_allowed,turnovers,pred_margin,pred_total,pred_pf,pred_pa
0,0.307692,0.285714,28.350000,9/10/2009,18,19,away,-3,PIT,234,...,3,TEN,3,320,357,2,-3.190663,40.712512,18.760925,21.951588
1,0.285714,0.307692,36.183333,9/10/2009,19,18,home,3,TEN,321,...,2,PIT,3,357,320,3,3.190663,40.712512,21.951588,18.760925
2,0.363636,0.400000,29.116667,9/13/2009,16,19,away,-12,ATL,163,...,0,MIA,26,259,281,4,3.213416,35.048928,19.131172,15.917756
3,0.400000,0.363636,30.883333,9/13/2009,19,16,home,12,MIA,213,...,4,ATL,26,281,259,0,-3.213416,35.048928,15.917756,19.131172
4,0.250000,0.333333,26.550000,9/13/2009,10,16,away,5,CIN,227,...,2,DEN,19,302,307,0,7.790480,35.456606,21.623543,13.833063


In [26]:
points_df = nfl[['team', 'points', 'pred_pf', 'opp', 'points_allowed', 'pred_pa']]
points_df

,team,points,pred_pf,opp,points_allowed,pred_pa
0,TEN,0,18.760925,PIT,3,21.951588
1,PIT,3,21.951588,TEN,0,18.760925
2,MIA,7,19.131172,ATL,19,15.917756
3,ATL,19,15.917756,MIA,7,19.131172
4,DEN,12,21.623543,CIN,7,13.833063
5,CIN,7,13.833063,DEN,12,21.623543
6,MIN,34,16.027337,CLE,20,17.427505
7,CLE,20,17.427505,MIN,34,16.027337
8,JAX,12,17.205921,IND,14,22.968528
9,IND,14,22.968528,JAX,12,17.205921


In [27]:
nfl.to_csv("nfl_prediction.csv", index = False)